## Headline based similarity on Articles

> **Index :- 18001149**

> I am Pandu, like **Data Analysis**

Generally, we assess **similarity** based on **distance**. If the **distance** is minimum then high **similarity** and if it is maximum then low **similarity**.
To calculate the **distance**, we need to represent the headline as a **d-dimensional** vector. Then we can find out the **similarity** based on the **distance** between vectors.

There are multiple methods to represent a **text** as **d-dimensional** vector like **Bag of words**, **TF-IDF method**, **Word2Vec embedding** etc. Each method has its own advantages and disadvantages. 

Let's see the feature representation of headline through all the methods one by one.

In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
# Below libraries are for text processing using NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances

from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [3]:
# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  

from sklearn.metrics import pairwise_distances
import copy
from IPython.display import clear_output

import warnings

from re import sub
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
plotly.offline.init_notebook_mode (connected = True)

import random
warnings.filterwarnings("ignore")

In [10]:
files_path="D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files"
read_files=glob.glob(os.path.join(files_path,"*.txt"))

In [7]:
read_files

['D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 1.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 2.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 3.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 4.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 5.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 6.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 7.txt',
 'D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 8.txt']

In [11]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# myfile = open("D:\\DC Universe\\Ucsc\\Fourth Year\\SCS4204 Data Analytics\\Assignments\\News Files\\doc 1.txt", "r")
# myline = myfile.readline()
# print(myline)

In [13]:
#Data Extracted into csv files for further analyzation

new_data1=pd.read_csv(read_files[0],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data2=pd.read_csv(read_files[1],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data3=pd.read_csv(read_files[2],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data4=pd.read_csv(read_files[3],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data5=pd.read_csv(read_files[4],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data6=pd.read_csv(read_files[5],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data7=pd.read_csv(read_files[6],error_bad_lines=False,header=None,delimiter = ' \t ')

new_data8=pd.read_csv(read_files[7],error_bad_lines=False,header=None,delimiter = ' \t ')

In [14]:
new_data1.set_axis(["Information"],axis=1,inplace=True)

In [15]:
# new_data1.dtypes
new_data1["Information"]=new_data1["Information"].astype('string')

## Headlines Storing in a Pandas dataset

In [112]:
News_head1 = ["Hurricane Gilbert Heads Toward Dominican Coast","IRA terrorist attack","McDonald's Opens First Restaurant in China"]
# News_head2 = ["IRA terrorist attack"]
# News_head3= ["McDonald's Opens First Restaurant in China"]

In [113]:
Head= pd.DataFrame(News_head1, columns=['Headline'])

In [114]:
Head['Headline']=Head['Headline'].astype('string')
Head.dtypes

Headline    string
dtype: object

> **Stopword tokenization usinh NLTK library**

In [212]:
# This function is to remove stopwords from a particular column and to tokenize it
def Stopword_tokenize(data,name):
      
    def getting(sen):
        example_sent = sen

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 

        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        filtered_sentence = [] 

        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
        return filtered_sentence
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x

## A)Using TF-IDF method

**TF-IDF** method is a weighted measure which gives more importance to less frequent words in a corpus. It assigns a weight to each term(word) in a document based on **Term frequency(TF)** and **inverse document frequency(IDF)**.

**TF(i,j)** = (# times word i appears in document j) / (# words in document j)

**IDF(i,D)** = log_e(#documents in the corpus D) / (#documents containing word i)

weight(i,j) = **TF(i,j)** x **IDF(i,D)**

So if a word occurs more number of times in a document but less number of times in all other documents then its **TF-IDF** value will be high.


## 1)For the First Document

In [16]:
new_data1.describe()

,Information
count,25
unique,25
top,The previous opening-day record for sales was ...
freq,1


In [17]:
new_data1

,Information
0,Thousands of queue-hardened Soviets on Wednesd...
1,The world's largest version of the landmark Am...
2,"The Soviets, bundled in fur coats and hats, se..."
3,The crush of customers was so intense the comp...
4,I only waited an hour and I think they served ...
5,"And it was only 10 rubles for all this, she sa..."
6,Big Macs were priced at 3.75 rubles and double...
7,The official exchange rate is 1.59 dollar per ...
8,Half the day's sales were donated to the Sovie...
9,"The restaurant, built by the company in a join..."


In [18]:
info1=new_data1['Information'].to_numpy()

In [19]:
info1_len=len(info1)

In [20]:
info1

array(["Thousands of queue-hardened Soviets on Wednesday cheerfully lined up to get a taste of ''gamburgers'', ''chizburgers'' and ''Filay-o-feesh'' sandwiches as McDonald's opened in the land of Lenin for the first time.",
       "The world's largest version of the landmark American fast-food chain rang up 30,000 meals on 27 cash registers, breaking the opening-day record for McDonald's worldwide, officials said.",
       'The Soviets, bundled in fur coats and hats, seemed unfazed, lining up before dawn outside the 700 seat restaurant, the first of 20 planned across the Soviet Union.',
       'The crush of customers was so intense the company stayed open until midnight, two hours later than planned.',
       'I only waited an hour and I think they served thousands before me, said a happy middle-aged woman who works at an aluminum plant.',
       "And it was only 10 rubles for all this, she said. I'm taking it back for the girls at the factory to try.",
       "Big Macs were priced at 

In [32]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
count1 = CountVectorizer()
word_count1=count1.fit_transform(info1)
print(word_count1)

  (0, 362)	1
  (0, 245)	3
  (0, 290)	1
  (0, 175)	1
  (0, 331)	1
  (0, 249)	1
  (0, 383)	1
  (0, 83)	1
  (0, 209)	1
  (0, 376)	1
  (0, 364)	1
  (0, 158)	1
  (0, 351)	1
  (0, 155)	1
  (0, 86)	1
  (0, 28)	1
  (0, 136)	1
  (0, 135)	1
  (0, 310)	1
  (0, 36)	1
  (0, 223)	1
  (0, 253)	1
  (0, 190)	1
  (0, 355)	2
  (0, 201)	1
  :	:
  (23, 388)	1
  (23, 148)	1
  (23, 39)	1
  (23, 403)	1
  (23, 189)	1
  (23, 397)	1
  (23, 101)	2
  (23, 138)	1
  (23, 344)	1
  (23, 278)	1
  (23, 48)	1
  (23, 384)	1
  (23, 62)	1
  (23, 358)	1
  (23, 242)	2
  (23, 396)	1
  (23, 239)	1
  (24, 364)	1
  (24, 359)	1
  (24, 67)	1
  (24, 148)	1
  (24, 239)	1
  (24, 408)	1
  (24, 240)	1
  (24, 270)	1


In [28]:
word_count1.shape

(25, 411)

In [29]:
print(word_count1.toarray())

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


In [30]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count1)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count1.get_feature_names(),columns=["IDF_Weights"])

In [36]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count1)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count1.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf.sort_values(by=['IDF_Weights'])

,IDF_Weights
the,1.213574
and,1.424883
to,1.619039
of,1.693147
for,1.693147
...,...
food,3.564949
folk,3.564949
flags,3.564949
gamburgers,3.564949


### Proceeding to the TF-IDF transformation.

In [37]:
#tfidf
tf_idf_vector=tfidf_transformer.transform(word_count1)
feature_names = count1.get_feature_names()

In [38]:
first_document_vector=tf_idf_vector[1]
df_tfifd= pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["TF-Idf"])

In [39]:
df_tfifd.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
the,0.217551
registers,0.213023
rang,0.213023
breaking,0.213023
cash,0.213023
food,0.213023
27,0.213023
30,0.213023
meals,0.213023
largest,0.213023


## 2)For the Second Document

In [40]:
new_data2.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data2["Information"]=new_data2["Information"].astype('string')

In [43]:
info2=new_data2['Information'].to_numpy()
info2

array(['Hurricane Gilbert, packing 110 mph winds and torrential rain, moved over this capital city today after skirting Puerto Rico, Haiti and the Dominican Republic.',
       'There were no immediate reports of casualties.',
       'Telephone communications were affected.',
       "Right now it's actually moving over Jamaica,said Bob Sheets, director of the National Hurricane Center in Miami.",
       "We've already had reports of 110 mph winds on the eastern tip.",
       "It looks like the eye is going to move lengthwise across that island, and they're going to bear the full brunt of this powerful hurricane, Sheets said.",
       'Forecasters say Gilbert was expected to lash Jamaica throughout the day and was on track to later strike the Cayman Islands, a small British dependency northwest of Jamaica.',
       "Meanwhile, Havana Radio reported today that 25,000 people were evacuated from Guantanamo Province on Cuba's southeastern coast as strong winds fanning out from Gilbert began 

In [44]:
count2 = CountVectorizer()
word_count2=count2.fit_transform(info2)
print(word_count2)

  (0, 147)	1
  (0, 124)	1
  (0, 237)	1
  (0, 3)	1
  (0, 204)	1
  (0, 370)	1
  (0, 23)	2
  (0, 343)	1
  (0, 264)	1
  (0, 201)	1
  (0, 234)	1
  (0, 333)	1
  (0, 56)	1
  (0, 68)	1
  (0, 341)	1
  (0, 14)	1
  (0, 301)	1
  (0, 262)	1
  (0, 278)	1
  (0, 132)	1
  (0, 328)	1
  (0, 91)	1
  (0, 275)	1
  (1, 331)	1
  (1, 360)	1
  :	:
  (31, 275)	1
  (31, 331)	1
  (31, 360)	1
  (31, 221)	2
  (31, 87)	1
  (31, 151)	3
  (31, 273)	1
  (31, 109)	1
  (31, 55)	1
  (31, 364)	1
  (31, 112)	1
  (31, 303)	1
  (31, 307)	1
  (31, 27)	1
  (31, 319)	1
  (31, 69)	1
  (31, 85)	1
  (31, 248)	1
  (31, 100)	1
  (31, 52)	1
  (31, 240)	1
  (31, 288)	1
  (31, 90)	1
  (31, 233)	1
  (31, 230)	1


In [45]:
word_count2.shape
print(word_count2.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [46]:
# info2_len=len(info2)
# info2
tfidf_transformer2=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer2.fit(word_count2)
df_idf2 = pd.DataFrame(tfidf_transformer2.idf_, index=count2.get_feature_names(),columns=["IDF_Weights"])

In [211]:
tfidf_transformer2=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer2.fit(word_count2)
df_idf2 = pd.DataFrame(tfidf_transformer2.idf_, index=count2.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf2.sort_values(by=['IDF_Weights'],ascending=False).head(20)

,IDF_Weights
000,3.80336
official,3.80336
overnight,3.80336
outages,3.80336
out,3.80336
our,3.80336
other,3.80336
order,3.80336
only,3.80336
office,3.80336


In [52]:
#tfidf
tf_idf_vector2=tfidf_transformer2.transform(word_count2)
feature_names2 = count2.get_feature_names()

In [53]:
second_document_vector=tf_idf_vector2[1]
df_tfifd2= pd.DataFrame(second_document_vector.T.todense(), index=feature_names2, columns=["TF-Idf"])

In [54]:
df_tfifd2.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
no,0.445322
casualties,0.445322
immediate,0.445322
there,0.397848
reports,0.397848
were,0.235531
of,0.181726
000,0.000000
power,0.000000
preparation,0.000000


## 3)For the Third Document

In [55]:
new_data3.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data3["Information"]=new_data3["Information"].astype('string')

In [56]:
info3=new_data3['Information'].to_numpy()
info3

array(['Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas.',
       'The storm was approaching from the southeast with sustained winds of 75 mph gusting to 92 mph.',
       'There is no need for alarm, Civil Defense Director Eugenio Cabral said in a television alert shortly before midnight Saturday.',
       "Cabral said residents of the province of Barahona should closely follow Gilbert's movement.",
       'An estimated 100,000 people live in the province, including 70,000 in the city of Barahona, about 125 miles west of Santo Domingo.',
       'Tropical Storm Gilbert formed in the eastern Caribbean and strengthened into a hurricane Saturday night.',
       'The National Hurricane Center in Miami reported its position at 2 a.m.',
       'Sunday at latitude 16.1 north, longitude 67.5 west, about 140 miles south of Ponce, Puerto Rico, and 200 miles southeast 

In [57]:
count3 = CountVectorizer()
word_count3=count3.fit_transform(info3)
print(word_count3)

  (0, 75)	1
  (0, 64)	1
  (0, 148)	1
  (0, 154)	1
  (0, 150)	2
  (0, 47)	1
  (0, 122)	1
  (0, 146)	1
  (0, 19)	2
  (0, 37)	1
  (0, 44)	1
  (0, 17)	1
  (0, 83)	1
  (0, 70)	1
  (0, 110)	1
  (0, 139)	1
  (0, 40)	1
  (0, 153)	1
  (0, 112)	1
  (0, 61)	1
  (0, 72)	2
  (0, 165)	1
  (0, 71)	1
  (0, 117)	1
  (0, 131)	1
  :	:
  (15, 150)	3
  (15, 141)	2
  (15, 159)	1
  (15, 106)	1
  (15, 59)	1
  (15, 138)	1
  (15, 100)	1
  (15, 7)	1
  (15, 25)	1
  (15, 132)	1
  (15, 133)	1
  (16, 75)	1
  (16, 150)	2
  (16, 40)	1
  (16, 27)	1
  (16, 55)	1
  (16, 43)	1
  (16, 118)	1
  (16, 95)	1
  (16, 142)	1
  (16, 28)	1
  (16, 73)	1
  (16, 91)	1
  (16, 85)	1
  (16, 96)	1


In [59]:
word_count3.shape
print(word_count3.toarray())

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [60]:
# info2_len=len(info2)
# info2
tfidf_transformer3=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer3.fit(word_count3)
df_idf3 = pd.DataFrame(tfidf_transformer3.idf_, index=count3.get_feature_names(),columns=["IDF_Weights"])

In [61]:
tfidf_transformer3=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer3.fit(word_count3)
df_idf3 = pd.DataFrame(tfidf_transformer3.idf_, index=count3.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf3.sort_values(by=['IDF_Weights'])

,IDF_Weights
the,1.251314
and,1.587787
of,1.693147
hurricane,1.944462
to,2.098612
...,...
happy,3.197225
had,3.197225
gusts,3.197225
gulf,3.197225


In [62]:
#tfidf
tf_idf_vector3=tfidf_transformer3.transform(word_count3)
feature_names3 = count3.get_feature_names()

In [63]:
third_document_vector=tf_idf_vector3[1]
df_tfifd3= pd.DataFrame(third_document_vector.T.todense(), index=feature_names3, columns=["TF-Idf"])

In [64]:
df_tfifd3.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
mph,0.450933
gusting,0.287878
75,0.287878
sustained,0.287878
approaching,0.287878
92,0.287878
southeast,0.225467
from,0.225467
with,0.225467
the,0.225336


## 4)For the Fourth Document

In [65]:
new_data4.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data4["Information"]=new_data4["Information"].astype('string')

In [66]:
new_data4.shape

(42, 1)

In [67]:
info4=new_data4['Information'].to_numpy()
info4

array(['An explosion today flattened a military barracks and tore through nearby homes, killing 11 people and injuring 22, police said.',
       'The IRA claimed responsibility for the blast.',
       'More than 100 rescue workers frantically dug through the rubble of a three-story building that collapsed at the Royal Marines School of Music near Deal.',
       'Stunned neighbors gathered outside homes that were damaged or destroyed.',
       'Chief Police Inspector Alan Butterfield of Kent, who who provided the casualty figures and coordinated the rescue effort, first reported that one person was missing but later said everyone was accounted for.',
       'He said many of the injured were seriously hurt.',
       'There was a terrific crash which reminded me of the Blitz.',
       'After that, the ceiling started to fall down around me, said pensioner Joan Betteridge.',
       'Defense Secretary Tom King, inspecting the wreckage, said, It is not yet absolutely confirmed that it is a b

In [69]:
count4 = CountVectorizer()
word_count4=count4.fit_transform(info4)
print(word_count4)

  (0, 25)	1
  (0, 119)	1
  (0, 341)	1
  (0, 129)	1
  (0, 215)	1
  (0, 46)	1
  (0, 26)	2
  (0, 344)	1
  (0, 338)	1
  (0, 227)	1
  (0, 158)	1
  (0, 193)	1
  (0, 1)	1
  (0, 255)	1
  (0, 170)	1
  (0, 6)	1
  (0, 260)	1
  (0, 283)	1
  (1, 329)	2
  (1, 175)	1
  (1, 82)	1
  (1, 275)	1
  (1, 131)	1
  (1, 56)	1
  (2, 338)	1
  :	:
  (39, 164)	1
  (39, 304)	1
  (39, 257)	1
  (39, 48)	1
  (39, 130)	1
  (40, 362)	1
  (40, 180)	1
  (40, 378)	1
  (40, 367)	1
  (41, 46)	1
  (41, 26)	1
  (41, 329)	2
  (41, 239)	2
  (41, 362)	2
  (41, 180)	1
  (41, 354)	1
  (41, 373)	1
  (41, 203)	1
  (41, 32)	2
  (41, 247)	1
  (41, 367)	1
  (41, 310)	1
  (41, 140)	1
  (41, 200)	1
  (41, 196)	1


In [70]:
# info2_len=len(info2)
# info2
tfidf_transformer4=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer4.fit(word_count4)
df_idf4 = pd.DataFrame(tfidf_transformer4.idf_, index=count4.get_feature_names(),columns=["IDF_Weights"])

In [72]:
tfidf_transformer4=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer4.fit(word_count4)
df_idf4 = pd.DataFrame(tfidf_transformer4.idf_, index=count4.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf4.sort_values(by=['IDF_Weights'])

,IDF_Weights
the,1.177681
and,1.870828
was,1.927987
of,1.927987
said,1.988611
...,...
frantically,4.068053
four,4.068053
force,4.068053
heavy,4.068053


In [73]:
#tfidf
tf_idf_vector4=tfidf_transformer4.transform(word_count4)
feature_names4 = count4.get_feature_names()

In [74]:
fourth_document_vector=tf_idf_vector4[1]
df_tfifd4= pd.DataFrame(fourth_document_vector.T.todense(), index=feature_names4, columns=["TF-Idf"])

In [75]:
df_tfifd4.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
claimed,0.502906
blast,0.452781
responsibility,0.452781
for,0.389631
ira,0.316967
the,0.291177
100,0.000000
plaster,0.000000
province,0.000000
provided,0.000000


## 5)For the Fifth Document

In [76]:
new_data5.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data5["Information"]=new_data5["Information"].astype('string')

In [78]:
info5=new_data5['Information'].to_numpy()
info5

array(['Neighbors were breakfasting, heading to work or asleep in bed when an explosion at a military barracks turned their homes to rubble and they were confronted with the sight of  bodies being carried away.',
       'There was a terrific crash which reminded me of the Blitz.',
       'After that, the ceiling started to fall down around me, said Joan Betteridge, a pensioner in the southern England town of Deal, where the blast at the Royal Marines School of Music occurred.',
       'The Irish Republican Army claimed reponsibilty for the explosion, which police said killed 11 people and injured 22.',
       'Nearby resident Sean Minnock said, I was asleep but woke up with a hell of a jolt, the bedroom ceiling fell in on me.',
       'I woke to find huge slabs of plaster on the bed and floor.',
       'From the wrecked, smoke-clouded barracks, I could hear terrified screams of agony.',
       'People started rushing about all over the place.',
       'It was horrible to watch and list

In [79]:
count5 = CountVectorizer()
word_count5=count5.fit_transform(info5)
print(word_count5)

  (0, 130)	1
  (0, 220)	2
  (0, 40)	1
  (0, 84)	1
  (0, 208)	2
  (0, 230)	1
  (0, 139)	1
  (0, 15)	1
  (0, 100)	1
  (0, 25)	1
  (0, 221)	1
  (0, 9)	1
  (0, 66)	1
  (0, 16)	1
  (0, 121)	1
  (0, 22)	1
  (0, 212)	1
  (0, 201)	1
  (0, 94)	1
  (0, 163)	1
  (0, 10)	1
  (0, 205)	1
  (0, 52)	1
  (0, 228)	1
  (0, 200)	1
  :	:
  (27, 104)	2
  (27, 213)	1
  (27, 42)	1
  (27, 191)	1
  (27, 132)	1
  (27, 2)	1
  (27, 145)	1
  (27, 103)	1
  (28, 200)	1
  (28, 204)	1
  (28, 141)	1
  (28, 86)	1
  (28, 103)	1
  (28, 118)	1
  (28, 170)	1
  (28, 20)	1
  (28, 214)	1
  (29, 208)	1
  (29, 66)	1
  (29, 204)	1
  (29, 217)	2
  (29, 168)	1
  (29, 78)	1
  (29, 23)	1
  (29, 174)	1


In [80]:
# info2_len=len(info2)
# info2
tfidf_transformer5=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer5.fit(word_count5)
df_idf5 = pd.DataFrame(tfidf_transformer5.idf_, index=count5.get_feature_names(),columns=["IDF_Weights"])

In [81]:
tfidf_transformer5=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer5.fit(word_count5)
df_idf5 = pd.DataFrame(tfidf_transformer5.idf_, index=count5.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf5.sort_values(by=['IDF_Weights'])

,IDF_Weights
the,1.215111
was,1.794930
of,1.869038
and,1.869038
said,2.131402
...,...
home,3.740840
homes,3.740840
horrible,3.740840
has,3.740840


In [82]:
#tfidf
tf_idf_vector5=tfidf_transformer5.transform(word_count5)
feature_names5 = count5.get_feature_names()

In [83]:
five_document_vector=tf_idf_vector5[1]
df_tfifd5= pd.DataFrame(five_document_vector.T.todense(), index=feature_names5, columns=["TF-Idf"])

In [84]:
df_tfifd5.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
crash,0.390942
blitz,0.390942
terrific,0.390942
reminded,0.390942
which,0.348568
me,0.318504
there,0.276130
of,0.195327
was,0.187582
the,0.126987


## 6)For the Sixth Document

In [85]:
new_data6.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data6["Information"]=new_data6["Information"].astype('string')

In [86]:
info6=new_data6['Information'].to_numpy()
info6

array(['An explosion rocked the Royal Marines School of Music in a southeastern coastal town today, causing one building to collapse and killing eight people, officials said.',
       'Thirty people were injured and up to 18 were missing and feared trapped in the rubble.',
       'The blast occurred at at 8:26 a.m. in a lounge in the barracks near Deal, about 70 miles southeast of London, the Defense Ministry said.',
       'The building has collapsed, said a ministry spokesman, speaking anonymously in keeping with British custom.',
       "We've no idea of the cause of the blast at the moment.",
       'It is too early to tell.',
       'Scotland Yard said a forensic team from its antiterrorist squad had been called in to help investigate.',
       'Firefighters used heavy lifting equipment and thermal cameras to search for those trapped in the debris, said Kent Fire Brigade spokesman Kevin Simmons.',
       'Kent police said 17 or 18 people were trapped.',
       'The Defense Ministr

In [88]:
count6 = CountVectorizer()
word_count6=count6.fit_transform(info6)
print(word_count6)

  (0, 14)	1
  (0, 76)	1
  (0, 165)	1
  (0, 195)	1
  (0, 166)	1
  (0, 130)	1
  (0, 171)	1
  (0, 147)	1
  (0, 138)	1
  (0, 101)	1
  (0, 185)	1
  (0, 52)	1
  (0, 209)	1
  (0, 205)	1
  (0, 48)	1
  (0, 150)	1
  (0, 38)	1
  (0, 204)	1
  (0, 53)	1
  (0, 15)	1
  (0, 120)	1
  (0, 67)	1
  (0, 156)	1
  (0, 148)	1
  (0, 169)	1
  :	:
  (18, 33)	1
  (18, 99)	1
  (18, 49)	1
  (18, 121)	1
  (18, 50)	1
  (18, 27)	1
  (18, 45)	1
  (18, 162)	1
  (18, 153)	1
  (18, 222)	1
  (18, 26)	1
  (18, 158)	1
  (19, 195)	1
  (19, 147)	1
  (19, 101)	1
  (19, 15)	1
  (19, 156)	1
  (19, 219)	1
  (19, 103)	1
  (19, 61)	1
  (19, 33)	1
  (19, 202)	1
  (19, 122)	1
  (19, 208)	1
  (19, 7)	1


In [89]:
# info2_len=len(info2)
# info2
tfidf_transformer6=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer6.fit(word_count6)
df_idf6 = pd.DataFrame(tfidf_transformer6.idf_, index=count6.get_feature_names(),columns=["IDF_Weights"])

In [90]:
tfidf_transformer6=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer6.fit(word_count6)
df_idf6 = pd.DataFrame(tfidf_transformer6.idf_, index=count6.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf6.sort_values(by=['IDF_Weights'],ascending=False).head(20)

,IDF_Weights
11,3.351375
owns,3.351375
military,3.351375
moment,3.351375
music,3.351375
narrowly,3.351375
nine,3.351375
no,3.351375
northern,3.351375
nothing,3.351375


In [91]:
#tfidf
tf_idf_vector6=tfidf_transformer6.transform(word_count6)
feature_names6 = count6.get_feature_names()

In [92]:
sixth_document_vector=tf_idf_vector6[1]
df_tfifd6= pd.DataFrame(sixth_document_vector.T.todense(), index=feature_names6, columns=["TF-Idf"])

In [93]:
sixth_document_vector

<1x230 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [94]:
df_tfifd6.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
were,0.385378
and,0.319880
feared,0.307714
rubble,0.307714
thirty,0.307714
up,0.307714
18,0.270486
missing,0.270486
trapped,0.244071
injured,0.244071


## 7)For the Seventh Document

In [95]:
new_data7.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data7["Information"]=new_data7["Information"].astype('string')

In [96]:
info7=new_data7['Information'].to_numpy()
info7

array(['Hurricane Gilbert slammed into Kingston on Monday with torrential rains and 115 mph winds that ripped roofs off homes and buildings, uprooted trees and downed power lines.',
       'No serious injuries were immediately reported in the city of 750,000 people, which was hit by the full force of the hurricane around noon.',
       'For half an hour, the hurricane lashed the city, tearing branches from trees, blowing down fences and whipping paper through the air.',
       "The National Weather Service reported heavy damage to Kingston's airport and aircraft parked on its fields.",
       'The first shock let up as the eye of the storm moved across the city.',
       'Skies brightened, the winds died down and people waited for an hour before the second blow of the hurricane arrived.',
       'All Jamaica-bound flights were canceled at Miami International Airport.',
       'Flights from the Cayman Islands, reportedly next in the path of the hurricane, arrived in Miami packed with tr

In [97]:
count7 = CountVectorizer()
word_count7=count7.fit_transform(info7)
print(word_count7)

  (0, 130)	1
  (0, 107)	1
  (0, 258)	1
  (0, 137)	1
  (0, 146)	1
  (0, 195)	1
  (0, 170)	1
  (0, 320)	1
  (0, 286)	1
  (0, 222)	1
  (0, 20)	3
  (0, 2)	1
  (0, 177)	1
  (0, 319)	1
  (0, 275)	1
  (0, 235)	1
  (0, 237)	1
  (0, 193)	1
  (0, 127)	1
  (0, 49)	1
  (0, 293)	1
  (0, 289)	1
  (0, 82)	1
  (0, 211)	1
  (0, 154)	1
  :	:
  (28, 171)	1
  (28, 217)	1
  (28, 72)	1
  (28, 11)	1
  (29, 107)	1
  (29, 195)	1
  (29, 132)	1
  (29, 276)	2
  (29, 192)	1
  (29, 101)	1
  (29, 196)	1
  (29, 54)	1
  (29, 103)	1
  (29, 116)	1
  (29, 106)	1
  (29, 269)	1
  (29, 76)	1
  (29, 60)	1
  (29, 149)	1
  (29, 272)	1
  (29, 308)	1
  (29, 299)	1
  (29, 246)	1
  (29, 158)	1
  (29, 283)	1


In [98]:
tfidf_transformer7=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer7.fit(word_count7)
df_idf7 = pd.DataFrame(tfidf_transformer7.idf_, index=count7.get_feature_names(),columns=["IDF_Weights"])

In [99]:
tfidf_transformer7=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer7.fit(word_count7)
df_idf7 = pd.DataFrame(tfidf_transformer7.idf_, index=count7.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf7.sort_values(by=['IDF_Weights'],ascending=False)

,IDF_Weights
man,3.740840
neighboring,3.740840
noon,3.740840
north,3.740840
not,3.740840
...,...
to,1.794930
hurricane,1.794930
and,1.661398
of,1.389465


In [101]:
#tfidf
tf_idf_vector7=tfidf_transformer7.transform(word_count7)
feature_names7 = count7.get_feature_names()

In [102]:
seven_document_vector=tf_idf_vector7[1]
df_tfifd7= pd.DataFrame(seven_document_vector.T.todense(), index=feature_names7, columns=["TF-Idf"])

In [103]:
df_tfifd7.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
serious,0.248001
hit,0.248001
immediately,0.248001
750,0.248001
noon,0.248001
force,0.248001
by,0.248001
the,0.233869
000,0.221121
no,0.221121


## 8)For the eighth Document

In [104]:
new_data8.set_axis(["Information"],axis=1,inplace=True)
# new_data1.dtypes
new_data8["Information"]=new_data8["Information"].astype('string')

In [105]:
info8=new_data8['Information'].to_numpy()
info8

array(["Communism suffered its first Big Mac attack Thursday as McDonald's opened a restaurant in Yugoslavia, and police were called in to keep customers who lined up for hours from getting too unruly under the golden arches.",
       'I just wanted to taste genuine American hamburgers, said Milica Nikolic, a high school student who waited for three hours to taste her first Big Mac.',
       "People curiously examined the renovated restaurant's plush interior and the back-lit signs depicting the hamburgers, french fries, milk shakes and other fare more familiar in the West.",
       'It also featured amber-colored tables and floors, pastel-colored upholstery, modern art paintings and discreet illumination.',
       'The fast-food outlet, located on a downtown square, had drawn crowds in recent days, and they began gathering long before it opened Thursday.',
       'Police kept watch on the lines of customers snaking around the block, and they regulated the number who came inside to avo

In [106]:
count8 = CountVectorizer()
word_count8=count8.fit_transform(info8)
print(word_count8)

  (0, 53)	1
  (0, 266)	1
  (0, 161)	1
  (0, 120)	1
  (0, 34)	1
  (0, 177)	1
  (0, 24)	1
  (0, 277)	1
  (0, 22)	1
  (0, 182)	1
  (0, 209)	1
  (0, 235)	1
  (0, 156)	2
  (0, 309)	1
  (0, 16)	1
  (0, 226)	1
  (0, 300)	1
  (0, 42)	1
  (0, 279)	1
  (0, 165)	1
  (0, 71)	1
  (0, 302)	1
  (0, 169)	1
  (0, 286)	1
  (0, 125)	1
  :	:
  (22, 236)	1
  (22, 63)	1
  (22, 223)	1
  (22, 208)	1
  (22, 121)	2
  (22, 7)	1
  (22, 194)	1
  (22, 307)	1
  (23, 120)	1
  (23, 182)	1
  (23, 156)	2
  (23, 16)	1
  (23, 279)	1
  (23, 271)	2
  (23, 159)	1
  (23, 66)	1
  (23, 208)	1
  (23, 194)	2
  (23, 91)	1
  (23, 102)	1
  (23, 260)	1
  (23, 35)	1
  (23, 191)	1
  (23, 37)	1
  (23, 153)	1


In [107]:
tfidf_transformer8=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer8.fit(word_count8)
df_idf8 = pd.DataFrame(tfidf_transformer8.idf_, index=count8.get_feature_names(),columns=["IDF_Weights"])

In [108]:
tfidf_transformer8=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer8.fit(word_count8)
df_idf8 = pd.DataFrame(tfidf_transformer8.idf_, index=count8.get_feature_names(),columns=["IDF_Weights"])

#inverse document frequency
df_idf8.sort_values(by=['IDF_Weights'],ascending=False).head(20)

,IDF_Weights
110,3.525729
numerous,3.525729
market,3.525729
meals,3.525729
media,3.525729
milk,3.525729
million,3.525729
milosevic,3.525729
modern,3.525729
month,3.525729


In [109]:
#tfidf
tf_idf_vector8=tfidf_transformer8.transform(word_count8)
feature_names8 = count8.get_feature_names()

In [110]:
eight_document_vector=tf_idf_vector8[1]
df_tfifd8= pd.DataFrame(eight_document_vector.T.todense(), index=feature_names8, columns=["TF-Idf"])

In [111]:
df_tfifd8.sort_values(by=["TF-Idf"],ascending=False).head(20)

,TF-Idf
taste,0.444788
student,0.222394
three,0.222394
waited,0.222394
genuine,0.222394
school,0.222394
wanted,0.222394
nikolic,0.222394
to,0.208651
milica,0.196818


### Recalling the Headlines

In [117]:
Head_len=len(Head)
Head_len

3

In [116]:
# to select multiple rows
result = Head.iloc[[0,1,2]]
result

,Headline
0,Hurricane Gilbert Heads Toward Dominican Coast
1,IRA terrorist attack
2,McDonald's Opens First Restaurant in China


## B) RapidFuzz

In [42]:
import rapidfuzz as rp
from rapidfuzz import process, fuzz

### Document 1 Testing

#### Indentify as Title 
> **McDonald's Opens First Restaurant in China**

## **Selecting Partial Ratio is provides the optimal results for the String Matching according to News Title**

**Partial Ratio**: It finds the ratio similarity measure between the shorter string and every substring of length m of the longer string, and returns the maximum of those similarity measures. Basically, it searches for the optimal alignment of the shorter string in the longer string and returns the fuzz.ratio for this

### Higher the Value Similaity of the text is increasing. Lower Score gives high chance of mismatches in the text.

In [179]:
info1 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(["Thousands of queue-hardened Soviets on Wednesday cheerfully lined up to get a taste of ''gamburgers'', ''chizburgers'' and ''Filay-o-feesh'' sandwiches as McDonald's opened in the land of Lenin for the first time.",
       "The world's largest version of the landmark American fast-food chain rang up 30,000 meals on 27 cash registers, breaking the opening-day record for McDonald's worldwide, officials said.",
       'The Soviets, bundled in fur coats and hats, seemed unfazed, lining up before dawn outside the 700 seat restaurant, the first of 20 planned across the Soviet Union.',
       'The crush of customers was so intense the company stayed open until midnight, two hours later than planned.',
       'I only waited an hour and I think they served thousands before me, said a happy middle-aged woman who works at an aluminum plant.',
       "And it was only 10 rubles for all this, she said. I'm taking it back for the girls at the factory to try.",
       "Big Macs were priced at 

In [162]:
 rp.fuzz.partial_ratio("McDonald's built its own factory, including bakery, dairy, meat processing plant and even potato storage yard, to provide its own guaranteed supplies in a country where up to 25 percent of the harvest rots en route to the consumer", "McDonald's Opens First Restaurant in China")

55.072463768115945

> Compare Other Article Titles ==> **Which provides Lower Values**

In [163]:
 rp.fuzz.partial_ratio("McDonald's built its own factory, including bakery, dairy, meat processing plant and even potato storage yard, to provide its own guaranteed supplies in a country where up to 25 percent of the harvest rots en route to the consumer", "Hurricane Gilbert Heads Toward Dominican Coast")

43.47826086956522

In [164]:
rp.fuzz.partial_ratio("McDonald's built its own factory, including bakery, dairy, meat processing plant and even potato storage yard, to provide its own guaranteed supplies in a country where up to 25 percent of the harvest rots en route to the consumer", "IRA terrorist attack")

44.99999999999999

#### Other titles  provide less with text similarity score which means our Selected title is the Correct One

### Document 2 Testing

#### Indentify as Title 
> **Hurricane Gilbert Heads Toward Dominican Coast**

In [143]:
info2 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['Hurricane Gilbert, packing 110 mph winds and torrential rain, moved over this capital city today after skirting Puerto Rico, Haiti and the Dominican Republic.',
       'There were no immediate reports of casualties.',
       'Telephone communications were affected.',
       "Right now it's actually moving over Jamaica,said Bob Sheets, director of the National Hurricane Center in Miami.",
       "We've already had reports of 110 mph winds on the eastern tip.",
       "It looks like the eye is going to move lengthwise across that island, and they're going to bear the full brunt of this powerful hurricane, Sheets said.",
       'Forecasters say Gilbert was expected to lash Jamaica throughout the day and was on track to later strike the Cayman Islands, a small British dependency northwest of Jamaica.',
       "Meanwhile, Havana Radio reported today that 25,000 people were evacuated from Guantanamo Province on Cuba's southeastern coast as strong winds fanning out from Gilbert began 

In [161]:
 rp.fuzz.partial_ratio("Hurricane Gilbert, packing 110 mph winds and torrential rain, moved over this capital city today after skirting Puerto Rico, Haiti and the Dominican Republic.", "Hurricane Gilbert Heads Toward Dominican Coast")

60.86956521739131

> Compare Other Article Titles ==> **Which provides Lower Values**

In [165]:
rp.fuzz.partial_ratio("Hurricane Gilbert, packing 110 mph winds and torrential rain, moved over this capital city today after skirting Puerto Rico, Haiti and the Dominican Republic.", "IRA terrorist attack")

50.0

In [166]:
 rp.fuzz.partial_ratio("Hurricane Gilbert, packing 110 mph winds and torrential rain, moved over this capital city today after skirting Puerto Rico, Haiti and the Dominican Republic.", "McDonald's Opens First Restaurant in China")

40.476190476190474

### Document 3 Testing

#### Indentify as Title 
> **Hurricane Gilbert Heads Toward Dominican Coast**

In [178]:
info3 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas.',
       'The storm was approaching from the southeast with sustained winds of 75 mph gusting to 92 mph.',
       'There is no need for alarm, Civil Defense Director Eugenio Cabral said in a television alert shortly before midnight Saturday.',
       "Cabral said residents of the province of Barahona should closely follow Gilbert's movement.",
       'An estimated 100,000 people live in the province, including 70,000 in the city of Barahona, about 125 miles west of Santo Domingo.',
       'Tropical Storm Gilbert formed in the eastern Caribbean and strengthened into a hurricane Saturday night.',
       'The National Hurricane Center in Miami reported its position at 2 a.m.',
       'Sunday at latitude 16.1 north, longitude 67.5 west, about 140 miles south of Ponce, Puerto Rico, and 200 miles southeast 

In [167]:
 rp.fuzz.partial_ratio("Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas.", "Hurricane Gilbert Heads Toward Dominican Coast")

79.12087912087912

> Compare Other Article Titles ==> **Which provides Lower Values**

In [168]:
rp.fuzz.partial_ratio("Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas.", "IRA terrorist attack")

40.0

In [169]:
 rp.fuzz.partial_ratio("Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas.", "McDonald's Opens First Restaurant in China")

42.85714285714286

### Document 4 Testing

#### Indentify as Title 
> **IRA terrorist attack**

In [174]:
info4 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['An explosion today flattened a military barracks and tore through nearby homes, killing 11 people and injuring 22, police said.',
       'The IRA claimed responsibility for the blast.',
       'More than 100 rescue workers frantically dug through the rubble of a three-story building that collapsed at the Royal Marines School of Music near Deal.',
       'Stunned neighbors gathered outside homes that were damaged or destroyed.',
       'Chief Police Inspector Alan Butterfield of Kent, who who provided the casualty figures and coordinated the rescue effort, first reported that one person was missing but later said everyone was accounted for.',
       'He said many of the injured were seriously hurt.',
       'There was a terrific crash which reminded me of the Blitz.',
       'After that, the ceiling started to fall down around me, said pensioner Joan Betteridge.',
       'Defense Secretary Tom King, inspecting the wreckage, said, It is not yet absolutely confirmed that it is a b

In [175]:
 rp.fuzz.partial_ratio("The IRA said Mrs Thatcher went to the British province with a message of war,but we still want peace and we want the British government to leave our country.", "IRA terrorist attack")

50.0

> Compare Other Article Titles ==> **Which provides Lower Values**

In [176]:
rp.fuzz.partial_ratio("The IRA said Mrs Thatcher went to the British province with a message of war,but we still want peace and we want the British government to leave our country.", "Hurricane Gilbert Heads Toward Dominican Coast")

41.30434782608695

In [177]:
 rp.fuzz.partial_ratio("The IRA said Mrs Thatcher went to the British province with a message of war,but we still want peace and we want the British government to leave our country.", "McDonald's Opens First Restaurant in China")

40.476190476190474

### Document 5 Testing

#### Indentify the as Title 
> **IRA terrorist attack**

In [180]:
info5 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['Neighbors were breakfasting, heading to work or asleep in bed when an explosion at a military barracks turned their homes to rubble and they were confronted with the sight of  bodies being carried away.',
       'There was a terrific crash which reminded me of the Blitz.',
       'After that, the ceiling started to fall down around me, said Joan Betteridge, a pensioner in the southern England town of Deal, where the blast at the Royal Marines School of Music occurred.',
       'The Irish Republican Army claimed reponsibilty for the explosion, which police said killed 11 people and injured 22.',
       'Nearby resident Sean Minnock said, I was asleep but woke up with a hell of a jolt, the bedroom ceiling fell in on me.',
       'I woke to find huge slabs of plaster on the bed and floor.',
       'From the wrecked, smoke-clouded barracks, I could hear terrified screams of agony.',
       'People started rushing about all over the place.',
       'It was horrible to watch and list

In [193]:
 rp.fuzz.partial_ratio("The IRA claimed reponsibilty for the explosion, which police said killed 11 people and injured 22", "IRA terrorist attack")

40.0

> Compare Other Article Titles ==> **Which provides Lower Values**

In [192]:
rp.fuzz.partial_ratio("The IRA claimed reponsibilty for the explosion, which police said killed 11 people and injured 22", "Hurricane Gilbert Heads Toward Dominican Coast")

37.2093023255814

In [196]:
 rp.fuzz.partial_ratio("The Irish Republican Army claimed reponsibilty for the explosion", "McDonald's Opens First Restaurant in China")

37.5

### Document 6 Testing

#### Indentify the as Title 
> **IRA terrorist attack**

In [197]:
info6 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['An explosion rocked the Royal Marines School of Music in a southeastern coastal town today, causing one building to collapse and killing eight people, officials said.',
       'Thirty people were injured and up to 18 were missing and feared trapped in the rubble.',
       'The blast occurred at at 8:26 a.m. in a lounge in the barracks near Deal, about 70 miles southeast of London, the Defense Ministry said.',
       'The building has collapsed, said a ministry spokesman, speaking anonymously in keeping with British custom.',
       "We've no idea of the cause of the blast at the moment.",
       'It is too early to tell.',
       'Scotland Yard said a forensic team from its antiterrorist squad had been called in to help investigate.',
       'Firefighters used heavy lifting equipment and thermal cameras to search for those trapped in the debris, said Kent Fire Brigade spokesman Kevin Simmons.',
       'Kent police said 17 or 18 people were trapped.',
       'The Defense Ministr

In [198]:
 rp.fuzz.partial_ratio("One soldier was killed and nine wounded in an IRA attack on an army barracks in north London in August 1988. About 60 soldiers narrowly escaped death or injury in February when they were evacuated from their barracks in Shropshire, western England, just before a bomb exploded.", "IRA terrorist attack")

50.0

> Compare Other Article Titles ==> **Which provides Lower Values**

In [199]:
rp.fuzz.partial_ratio("One soldier was killed and nine wounded in an IRA attack on an army barracks in north London in August 1988. About 60 soldiers narrowly escaped death or injury in February when they were evacuated from their barracks in Shropshire, western England, just before a bomb exploded.", "Hurricane Gilbert Heads Toward Dominican Coast")

44.99999999999999

In [200]:
 rp.fuzz.partial_ratio("One soldier was killed and nine wounded in an IRA attack on an army barracks in north London in August 1988. About 60 soldiers narrowly escaped death or injury in February when they were evacuated from their barracks in Shropshire, western England, just before a bomb exploded.", "McDonald's Opens First Restaurant in China")

42.85714285714286

### Document 7 Testing

#### Indentify the as Title 
> **Hurricane Gilbert Heads Toward Dominican Coast**

In [202]:
info7 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(['Hurricane Gilbert slammed into Kingston on Monday with torrential rains and 115 mph winds that ripped roofs off homes and buildings, uprooted trees and downed power lines.',
       'No serious injuries were immediately reported in the city of 750,000 people, which was hit by the full force of the hurricane around noon.',
       'For half an hour, the hurricane lashed the city, tearing branches from trees, blowing down fences and whipping paper through the air.',
       "The National Weather Service reported heavy damage to Kingston's airport and aircraft parked on its fields.",
       'The first shock let up as the eye of the storm moved across the city.',
       'Skies brightened, the winds died down and people waited for an hour before the second blow of the hurricane arrived.',
       'All Jamaica-bound flights were canceled at Miami International Airport.',
       'Flights from the Cayman Islands, reportedly next in the path of the hurricane, arrived in Miami packed with tr

In [203]:
rp.fuzz.partial_ratio("Hurricane Gilbert slammed into Kingston on Monday with torrential rains and 115 mph winds that ripped roofs off homes and buildings, uprooted trees and downed power lines.", "Hurricane Gilbert Heads Toward Dominican Coast")

67.46987951807229

> Compare Other Article Titles ==> **Which provides Lower Values**

In [204]:
 rp.fuzz.partial_ratio("Hurricane Gilbert slammed into Kingston on Monday with torrential rains and 115 mph winds that ripped roofs off homes and buildings, uprooted trees and downed power lines.", "IRA terrorist attack")

44.99999999999999

In [205]:
 rp.fuzz.partial_ratio("Hurricane Gilbert slammed into Kingston on Monday with torrential rains and 115 mph winds that ripped roofs off homes and buildings, uprooted trees and downed power lines..", "McDonald's Opens First Restaurant in China")

42.85714285714286

### Document 8 Testing

#### Indentify the as Title 
> **McDonald's Opens First Restaurant in China**

In [207]:
info8 #text will be extracted from the text files Respectivly for testing to verify our prediction is correct

array(["Communism suffered its first Big Mac attack Thursday as McDonald's opened a restaurant in Yugoslavia, and police were called in to keep customers who lined up for hours from getting too unruly under the golden arches.",
       'I just wanted to taste genuine American hamburgers, said Milica Nikolic, a high school student who waited for three hours to taste her first Big Mac.',
       "People curiously examined the renovated restaurant's plush interior and the back-lit signs depicting the hamburgers, french fries, milk shakes and other fare more familiar in the West.",
       'It also featured amber-colored tables and floors, pastel-colored upholstery, modern art paintings and discreet illumination.',
       'The fast-food outlet, located on a downtown square, had drawn crowds in recent days, and they began gathering long before it opened Thursday.',
       'Police kept watch on the lines of customers snaking around the block, and they regulated the number who came inside to avo

In [208]:
 rp.fuzz.partial_ratio("The next East European McDonald's, and the first in a Soviet bloc country, is to open next month in Budapest, Hungary", "McDonald's Opens First Restaurant in China")

52.38095238095239

> Compare Other Article Titles ==> **Which provides Lower Values**

In [209]:
rp.fuzz.partial_ratio("The next East European McDonald's, and the first in a Soviet bloc country, is to open next month in Budapest, Hungary", "Hurricane Gilbert Heads Toward Dominican Coast")

41.860465116279066

In [210]:
 rp.fuzz.partial_ratio("The next East European McDonald's, and the first in a Soviet bloc country, is to open next month in Budapest, Hungary", "IRA terrorist attack")

40.0

> **According to this results Lets Validate the Prediction with Cosin-Similarity**

## C) Finding Cosin Similarity

### Topics that content needs to identify
### Verifying Previous 2 methods results
> Hurricane Gilbert Heads Toward Dominican Coast =>> **Head.iloc[0]**

> IRA terrorist attack =>>  **Head.iloc[1]**

> McDonald's Opens First Restaurant in China **Head.iloc[2]**

#### Creating Vectorize Vocabulary to identify common Words

In [214]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [215]:
# opening the text files and copying to variables
def text_string(file_name):
    text = ''
    with open(file_name,"r") as provided_file:
        for line in provided_file:
   
            # reading each word        
            for word in line.split():
                text = text + word + ' '  
        
        return text.strip()

In [216]:
news_1 = text_string(read_files[0])
news_2 = text_string(read_files[1])
news_3 = text_string(read_files[2])
news_4 = text_string(read_files[3])
news_5 = text_string(read_files[4])
news_6 = text_string(read_files[5])
news_7 = text_string(read_files[6])
news_8 = text_string(read_files[7])

In [217]:
# create a corpus by using assigned variables
News_corpus = [news_1, news_2, news_3 ,news_4, news_5, news_6, news_7, news_8]

In [229]:
stop_words=stopwords.words('english')

In [232]:
import nltk
import string

In [233]:
def preprocessing(News_corpus):
  vec = []
  for word in News_corpus:                     #Examines individual words within the document array
    if(word not in stop_words):             #The word is discarded if it's a stop word
      if(word not in string.punctuation):   #If the read word is a special character it's discarded
        word = word.lower()                 #Coverts to lowercase

        vec.append(word);
  return vec     

In [236]:
for i in range(8):
  #print("\nDocument 0"+str(i+1))
  preprocessing(News_corpus[i])
  print(News_corpus[i])

Thousands of queue-hardened Soviets on Wednesday cheerfully lined up to get a taste of ''gamburgers'', ''chizburgers'' and ''Filay-o-feesh'' sandwiches as McDonald's opened in the land of Lenin for the first time. The world's largest version of the landmark American fast-food chain rang up 30,000 meals on 27 cash registers, breaking the opening-day record for McDonald's worldwide, officials said. The Soviets, bundled in fur coats and hats, seemed unfazed, lining up before dawn outside the 700 seat restaurant, the first of 20 planned across the Soviet Union. The crush of customers was so intense the company stayed open until midnight, two hours later than planned. I only waited an hour and I think they served thousands before me, said a happy middle-aged woman who works at an aluminum plant. And it was only 10 rubles for all this, she said. I'm taking it back for the girls at the factory to try. Big Macs were priced at 3.75 rubles and double cheeseburgers at 3 rubles about two hours' pa

In [244]:
def build_corpus(files):
  corpus = []
  for file in files:
    tokenized_words = word_tokenize(file)                 #Tokenize the sentences into words
    preprocessed_words = preprocessing(tokenized_words)   #Passes these individual words through the preprocessing function
    corpus.append(' '.join(preprocessed_words))           #Rejions individual words to sentences
  
  return corpus

In [251]:
#corpus = build_corpus(files) 


In [237]:
News_vectorizer = TfidfVectorizer()
vectors = News_vectorizer.fit_transform(News_corpus)

feature_names = News_vectorizer.get_feature_names_out()
print(feature_names, len(feature_names))


['000' '10' '100' ... 'yugoslavia' 'yugoslavs' 'zone'] 1411


#### Creating Pandas frame for denselist

In [238]:
for text in feature_names:
    print(text)

dense = vectors.todense()
denselist = dense.tolist()
News_df = pd.DataFrame(denselist, columns=feature_names)

000
10
100
11
110
115
12
125
14
140
15
16
17
18
1982
1988
20
200
22
25
250
26
27
30
350
50
500
51
52
57
59
60
67
70
700
72
75
750
80
90
92
ability
about
absolutely
accented
acceptance
accordions
accounted
accustomed
across
actions
actually
adding
additional
advised
affected
after
against
aged
agencies
agency
ago
agony
agreement
aim
air
aircraft
airport
alan
alarm
alcohol
alert
alerted
alex
alien
all
along
already
also
aluminum
am
amber
ambulance
ambulances
america
american
an
and
anonymously
another
antiterrorist
any
anything
apart
appeals
appeared
appears
approaching
approximating
arches
are
area
areas
arms
army
around
arrived
art
as
asked
asleep
assistance
associate
associated
at
atlantic
atrocity
attack
attacked
attempt
august
average
avoid
awaited
away
baba
back
backing
bag
bakery
balkan
band
bands
bandsmen
bandstand
bang
barahona
barracks
based
bases
bay
be
beaches
bear
bearing
because
bed
bedroom
beeg
been
before
began
behind
being
belgrade
ben
besides
betteridge
between
big
bits

In [239]:
News_df.dtypes

000           float64
10            float64
100           float64
11            float64
110           float64
               ...   
youthful      float64
yugoslav      float64
yugoslavia    float64
yugoslavs     float64
zone          float64
Length: 1411, dtype: object

In [240]:
News_df.head(20)

,000,10,100,11,110,115,12,125,14,140,...,year,years,yet,you,young,youthful,yugoslav,yugoslavia,yugoslavs,zone
0,0.052621,0.060016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055325,0.000000,...,0.000000,0.035081,0.000000,0.023184,0.046367,0.027663,0.000000,0.000000,0.000000,0.000000
1,0.015023,0.017134,0.034269,0.000000,0.059569,0.000000,0.000000,0.000000,0.000000,0.039713,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023693
2,0.061567,0.000000,0.035110,0.000000,0.000000,0.000000,0.048548,0.048548,0.000000,0.040687,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.017594,0.035188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015426,0.024329,0.000000,0.040778,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.027831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.024401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.031240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.043197,0.000000,0.000000,0.036202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.033987,0.019382,0.000000,0.000000,0.000000,0.053601,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.025177,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.019049,0.000000,0.000000,0.000000,0.000000,0.120167,0.090126,0.030042,0.000000


> **Count of Total Vocabulary**

In [241]:
print(len(News_vectorizer.vocabulary_))

1411


In [242]:
News_vectorizer.vocabulary_

{'thousands': 1266,
 'of': 871,
 'queue': 997,
 'hardened': 575,
 'soviets': 1176,
 'on': 877,
 'wednesday': 1354,
 'cheerfully': 261,
 'lined': 722,
 'up': 1314,
 'to': 1275,
 'get': 537,
 'taste': 1235,
 'gamburgers': 527,
 'chizburgers': 267,
 'and': 87,
 'filay': 475,
 'feesh': 467,
 'sandwiches': 1077,
 'as': 108,
 'mcdonald': 770,
 'opened': 883,
 'in': 636,
 'the': 1251,
 'land': 697,
 'lenin': 714,
 'for': 502,
 'first': 483,
 'time': 1272,
 'world': 1388,
 'largest': 701,
 'version': 1329,
 'landmark': 698,
 'american': 85,
 'fast': 460,
 'food': 501,
 'chain': 252,
 'rang': 1003,
 '30': 23,
 '000': 0,
 'meals': 773,
 '27': 22,
 'cash': 236,
 'registers': 1019,
 'breaking': 185,
 'opening': 884,
 'day': 347,
 'record': 1014,
 'worldwide': 1389,
 'officials': 875,
 'said': 1074,
 'bundled': 207,
 'fur': 524,
 'coats': 284,
 'hats': 578,
 'seemed': 1103,
 'unfazed': 1309,
 'lining': 724,
 'before': 150,
 'dawn': 346,
 'outside': 898,
 '700': 34,
 'seat': 1098,
 'restaurant': 104

In [243]:
# get cosine similarity matrix by using created dataframe
print(cosine_similarity(News_df.values, News_df.values))

[[1.         0.54082435 0.42279101 0.56849572 0.50821448 0.48433927
  0.5472568  0.59938462]
 [0.54082435 1.         0.68250002 0.61184088 0.53099415 0.53330571
  0.84194483 0.50997942]
 [0.42279101 0.68250002 1.         0.4670546  0.40854181 0.41944037
  0.65355313 0.3833241 ]
 [0.56849572 0.61184088 0.4670546  1.         0.74806744 0.7313034
  0.61104358 0.54061195]
 [0.50821448 0.53099415 0.40854181 0.74806744 1.         0.55023635
  0.53310933 0.45222212]
 [0.48433927 0.53330571 0.41944037 0.7313034  0.55023635 1.
  0.52849343 0.46656382]
 [0.5472568  0.84194483 0.65355313 0.61104358 0.53310933 0.52849343
  1.         0.51410949]
 [0.59938462 0.50997942 0.3833241  0.54061195 0.45222212 0.46656382
  0.51410949 1.        ]]


In [227]:
Document_similar = cosine_similarity(News_df.values, News_df.values)
Doc_similar = pd.DataFrame(Document_similar, columns = ['Doc1','Doc2','Doc3','Doc4','Doc5','Doc6','Doc7','Doc8'])

In [228]:
Doc_similar

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8
0,1.000000,0.540824,0.422791,0.568496,0.508214,0.484339,0.547257,0.599385
1,0.540824,1.000000,0.682500,0.611841,0.530994,0.533306,0.841945,0.509979
2,0.422791,0.682500,1.000000,0.467055,0.408542,0.419440,0.653553,0.383324
3,0.568496,0.611841,0.467055,1.000000,0.748067,0.731303,0.611044,0.540612
4,0.508214,0.530994,0.408542,0.748067,1.000000,0.550236,0.533109,0.452222
5,0.484339,0.533306,0.419440,0.731303,0.550236,1.000000,0.528493,0.466564
6,0.547257,0.841945,0.653553,0.611044,0.533109,0.528493,1.000000,0.514109
7,0.599385,0.509979,0.383324,0.540612,0.452222,0.466564,0.514109,1.000000


In [253]:
Doc_similar.sort_values(by=['Doc1'],ascending=False)

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8
0,1.000000,0.540824,0.422791,0.568496,0.508214,0.484339,0.547257,0.599385
7,0.599385,0.509979,0.383324,0.540612,0.452222,0.466564,0.514109,1.000000
3,0.568496,0.611841,0.467055,1.000000,0.748067,0.731303,0.611044,0.540612
6,0.547257,0.841945,0.653553,0.611044,0.533109,0.528493,1.000000,0.514109
1,0.540824,1.000000,0.682500,0.611841,0.530994,0.533306,0.841945,0.509979
4,0.508214,0.530994,0.408542,0.748067,1.000000,0.550236,0.533109,0.452222
5,0.484339,0.533306,0.419440,0.731303,0.550236,1.000000,0.528493,0.466564
2,0.422791,0.682500,1.000000,0.467055,0.408542,0.419440,0.653553,0.383324


# Additional Work done for the Similarity Checking

## ---------------------------------------------------------------------------------------------------

In [150]:
new_data1.iloc[0]

Information    Thousands of queue-hardened Soviets on Wednesd...
Name: 0, dtype: string

In [157]:
new_data1.values[0]

array(["Thousands of queue-hardened Soviets on Wednesday cheerfully lined up to get a taste of ''gamburgers'', ''chizburgers'' and ''Filay-o-feesh'' sandwiches as McDonald's opened in the land of Lenin for the first time."],
      dtype=object)

In [193]:
new_data1.head(10)

,Information
0,Thousands of queue-hardened Soviets on Wednesd...
1,The world's largest version of the landmark Am...
2,"The Soviets, bundled in fur coats and hats, se..."
3,The crush of customers was so intense the comp...
4,I only waited an hour and I think they served ...
5,"And it was only 10 rubles for all this, she sa..."
6,Big Macs were priced at 3.75 rubles and double...
7,The official exchange rate is 1.59 dollar per ...
8,Half the day's sales were donated to the Sovie...
9,"The restaurant, built by the company in a join..."


In [194]:
r1=tfidf_doc1.fit_transform(new_data1.iloc[0],Head.iloc[0])
r1.sum()

5.059644256269405

In [197]:
for i in range(len(new_data1)):
    r1=tfidf_doc1.fit_transform(new_data1.iloc[i],Head.iloc[0])
    print(r1)
    print(r1.sum())
 
# print(r1_tot)
    
#     r1=tfidf_doc1.fit_transform(new_data1.iloc[i],Head.iloc[0])
#     df2 = pd.DataFrame({'info':new_data1['Information'].values[i],
#                        'Cosine Similarity':Head.iloc[0]})

  (0, 25)	0.15811388300841897
  (0, 6)	0.15811388300841897
  (0, 7)	0.15811388300841897
  (0, 13)	0.15811388300841897
  (0, 12)	0.15811388300841897
  (0, 23)	0.31622776601683794
  (0, 11)	0.15811388300841897
  (0, 18)	0.15811388300841897
  (0, 15)	0.15811388300841897
  (0, 1)	0.15811388300841897
  (0, 20)	0.15811388300841897
  (0, 4)	0.15811388300841897
  (0, 5)	0.15811388300841897
  (0, 0)	0.15811388300841897
  (0, 3)	0.15811388300841897
  (0, 8)	0.15811388300841897
  (0, 22)	0.15811388300841897
  (0, 9)	0.15811388300841897
  (0, 26)	0.15811388300841897
  (0, 27)	0.15811388300841897
  (0, 14)	0.15811388300841897
  (0, 2)	0.15811388300841897
  (0, 28)	0.15811388300841897
  (0, 17)	0.15811388300841897
  (0, 21)	0.15811388300841897
  (0, 10)	0.15811388300841897
  (0, 19)	0.15811388300841897
  (0, 16)	0.4743416490252569
  (0, 24)	0.15811388300841897
5.059644256269405
  (0, 22)	0.16666666666666666
  (0, 16)	0.16666666666666666
  (0, 27)	0.16666666666666666
  (0, 13)	0.16666666666666666
  (

In [200]:
# for i in range(len(new_data1)):
#     r2=tfidf_doc1.fit_transform(new_data1.iloc[i],Head.iloc[2])
#     print(r2)
#     print(r2.sum())

In [50]:
News_head1 = ["Hurricane Gilbert Heads Toward Dominican Coast"]
News_head2 = ["IRA terrorist attack"]
News_head3= ["McDonald's Opens First Restaurant in China"]
  
# Create the pandas DataFrame with column name is provided explicitly
Headlines1 = pd.DataFrame(News_head1, columns=['Headline'])
Headlines2 = pd.DataFrame(News_head2, columns=['Headline'])
Headlines3 = pd.DataFrame(News_head3, columns=['Headline'])

### Text file1 headline Revealing

In [51]:
Head1_title1=Headlines1.append([Headlines1]*24,ignore_index=True)
Head1_title2=Headlines2.append([Headlines2]*24,ignore_index=True)
Head1_title3=Headlines3.append([Headlines3]*24,ignore_index=True)

In [52]:
# Merge default pandas DataFrame without any key column
Head1_title1 = pd.concat([Head1_title1,new_data1], join = 'outer', axis = 1)
Head1_title2 = pd.concat([Head1_title2,new_data1], join = 'outer', axis = 1)
Head1_title3 = pd.concat([Head1_title3,new_data1], join = 'outer', axis = 1)

In [53]:
Head1_title3.head(5)

,Headline,Information
0,McDonald's Opens First Restaurant in China,"Thousands queue-hardened Soviets Wednesday cheerfully lined get taste `` gamburgers '' , `` chizburgers '' `` Filay-o-feesh '' sandwich McDonald 's opened land Lenin first time ."
1,McDonald's Opens First Restaurant in China,"The world 's large version landmark American fast-food chain rang 30,000 meal 27 cash register , breaking opening-day record McDonald 's worldwide , official said ."
2,McDonald's Opens First Restaurant in China,"The Soviets , bundled fur coat hat , seemed unfazed , lining dawn outside 700 seat restaurant , first 20 planned across Soviet Union ."
3,McDonald's Opens First Restaurant in China,"The crush customer intense company stayed open midnight , two hour late planned ."
4,McDonald's Opens First Restaurant in China,"I waited hour I think served thousand , said happy middle-aged woman work aluminum plant ."


In [54]:
frames = [Head1_title1 , Head1_title2 , Head1_title3]
Doc1 = pd.concat(frames)

In [55]:
Doc1.shape
Doc1

,Headline,Information
0,Hurricane Gilbert Heads Toward Dominican Coast,"Thousands queue-hardened Soviets Wednesday cheerfully lined get taste `` gamburgers '' , `` chizburgers '' `` Filay-o-feesh '' sandwich McDonald 's opened land Lenin first time ."
1,Hurricane Gilbert Heads Toward Dominican Coast,"The world 's large version landmark American fast-food chain rang 30,000 meal 27 cash register , breaking opening-day record McDonald 's worldwide , official said ."
2,Hurricane Gilbert Heads Toward Dominican Coast,"The Soviets , bundled fur coat hat , seemed unfazed , lining dawn outside 700 seat restaurant , first 20 planned across Soviet Union ."
3,Hurricane Gilbert Heads Toward Dominican Coast,"The crush customer intense company stayed open midnight , two hour late planned ."
4,Hurricane Gilbert Heads Toward Dominican Coast,"I waited hour I think served thousand , said happy middle-aged woman work aluminum plant ."
...,...,...
20,McDonald's Opens First Restaurant in China,"McDonald 's Canada Chairman George Cohon , man behind deal , said many people buying multiple order restaurant served 15,000 20,000 people first five hour operation ."
21,McDonald's Opens First Restaurant in China,The restaurant limited purchase 10 Big Macs per customer hope preventing burger scalping .
22,McDonald's Opens First Restaurant in China,"McDonald 's built factory , including bakery , dairy , meat processing plant even potato storage yard , provide guaranteed supply country 25 percent harvest rot en route consumer ."
23,McDonald's Opens First Restaurant in China,"One McDonald 's associate said company wound importing wooden crate Finland storing potato went build crate , found wood , nail ."


In [56]:
Doc1['Headline'] = Doc1['Headline'].str.replace('\d+', '',regex=True)

In [59]:
Doc1[['Headline','Information']] = Doc1[['Headline','Information']].astype('string')

In [106]:
Doc1.dtypes

Headline       string
Information    string
dtype: object

In [107]:
Doc1.head(5)

,Headline,Information
0,Hurricane Gilbert Heads Toward Dominican Coast,"Thousands queue-hardened Soviets Wednesday cheerfully lined get taste `` gamburgers '' , `` chizburgers '' `` Filay-o-feesh '' sandwich McDonald 's opened land Lenin first time ."
1,Hurricane Gilbert Heads Toward Dominican Coast,"The world 's large version landmark American fast-food chain rang 30,000 meal 27 cash register , breaking opening-day record McDonald 's worldwide , official said ."
2,Hurricane Gilbert Heads Toward Dominican Coast,"The Soviets , bundled fur coat hat , seemed unfazed , lining dawn outside 700 seat restaurant , first 20 planned across Soviet Union ."
3,Hurricane Gilbert Heads Toward Dominican Coast,"The crush customer intense company stayed open midnight , two hour late planned ."
4,Hurricane Gilbert Heads Toward Dominican Coast,"I waited hour I think served thousand , said happy middle-aged woman work aluminum plant ."


In [101]:
headline_vectorizer = CountVectorizer()
# headline_features   = headline_vectorizer.fit_transform(Doc1['Headline'])

In [80]:
headline_features.get_shape()

(75, 15)

### Using Bag of Words method

> Bag of Word model not provides the expected level of accuracy for similarity checking and it's being neglected

A **Bag of Words(BoW)** method represents the occurence of words within a **document**. Here, each headline can be considered as a **document** and set of all headlines form a **corpus**.

Using **BoW** approach, each **document** is represented by a **d-dimensional** vector, where **d** is total number of **unique words** in the corpus. The set of such unique words forms the **Vocabulary**.

In [99]:
# def bag_of_words_based_model(row_index, num_similar_items):
#     couple_dist = pairwise_distances(headline_features,headline_features[row_index])
#     indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
#     df = pd.DataFrame({'Information':Head1_title1['Information'][indices].values,
#                 'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
#     print("="*30,"Queried article details","="*30)
#     print('headline : ',Doc1['Headline'][indices[1]])
#     print("\n","="*25,"Recommended articles : ","="*23)
#     #return df.iloc[1:,1]
#     return df.iloc[1:,]

# # bag_of_words_based_model(20, 10) # Change the row index for any other queried article

# name=input('News Title For Recommendation :')
# clear_output()
# ind=Doc1[Doc1['Headline']==name].index[0]
# dd=bag_of_words_based_model(ind, 20)
# dd.head(10)  # Change the row index for any other queried article

In [39]:
text1 = new_data1.to_numpy()
process.extract("Hurricane Gilbert Heads Toward Dominican Coast",  new_data1, scorer=fuzz.ratio)

[(0     Thousands of queue-hardened Soviets on Wednesday cheerfully lined up to get a taste of ''gamburgers'', ''chizburgers'' and ''Filay-o-feesh'' sandwiches as McDonald's opened in the land of Lenin for the first time.                                                                                                  
  1     The world's largest version of the landmark American fast-food chain rang up 30,000 meals on 27 cash registers, breaking the opening-day record for McDonald's worldwide, officials said.                                                                                                                              
  2     The Soviets, bundled in fur coats and hats, seemed unfazed, lining up before dawn outside the 700 seat restaurant, the first of 20 planned across the Soviet Union.                                                                                                                                                    
  3     The crush of customers was so in